In [9]:
import pandas as pd
import json

def json_to_csv_override(year,path):
    with open('..\\data\\raw\\nvdcve_json\\nvdcve-1.1-'+year+'.json','r',encoding='utf-8') as f:
        data = json.load(f)
    list_y = data['CVE_Items']
    cve_cwe_list = []
    cve_id_list = []
    cve_des_list = []
    for cve in list_y:
        cwe_value_pre = cve['cve']['problemtype']['problemtype_data'][0]['description']
        if len(cwe_value_pre)==0:
            continue
        cve_id_list.append(cve['cve']['CVE_data_meta']['ID'])
        for value in cwe_value_pre:
            cwe_value = value
        cve_cwe_list.append(cwe_value['value'])
        cve_des_list.append(cve['cve']['description']['description_data'][0]['value'])
    cve_dic = {'cve_id':cve_id_list,'cwe_id':cve_cwe_list,'descript':cve_des_list}
    df = pd.DataFrame(cve_dic)
    df = df[~df['cwe_id'].str.contains('noinfo')]
    return df
#     df.to_csv(path,header=False,index = False)

def json_to_csv_append(year,path):
    with open('..\\data\\raw\\nvdcve_json\\nvdcve-1.1-'+year+'.json','r',encoding='utf-8') as f:
        data = json.load(f)
    list_y = data['CVE_Items']
    cve_cwe_list = []
    cve_id_list = []
    cve_des_list = []
    for cve in list_y:
        cwe_value_pre = cve['cve']['problemtype']['problemtype_data'][0]['description']
        # if len(cwe_value_pre)==0:
        #     continue
        if cwe_value_pre == []:
            continue
        cve_id_list.append(cve['cve']['CVE_data_meta']['ID'])
        for value in cwe_value_pre:
            cwe_value = value
        cve_cwe_list.append(cwe_value['value'])
        cve_des_list.append(cve['cve']['description']['description_data'][0]['value'])
    cve_dic = {'cve_id':cve_id_list,'cwe_id':cve_cwe_list,'descript':cve_des_list}
    df = pd.DataFrame(cve_dic)
    df = df[~df['cwe_id'].str.contains('noinfo')]
    return df
#     df.to_csv(path,header=False,index = False,mode='a')


In [20]:
year = '2006'
path = '..\\data\\clean\\nvdcve-1.1-'+year+'.csv'
df = json_to_csv_append(year,path)
df

,cve_id,cwe_id,descript
0,CVE-2006-0001,CWE-119,Stack-based buffer overflow in Microsoft Publi...
3,CVE-2006-0004,NVD-CWE-Other,Microsoft PowerPoint 2000 in Office 2000 SP3 h...
4,CVE-2006-0005,CWE-119,Buffer overflow in the plug-in for Microsoft W...
5,CVE-2006-0006,CWE-119,Heap-based buffer overflow in the bitmap proce...
6,CVE-2006-0007,CWE-119,"Buffer overflow in GIFIMP32.FLT, as used in Mi..."
...,...,...,...
6987,CVE-2006-7247,CWE-89,SQL injection vulnerability in the Weblinks (c...
6988,CVE-2006-7250,NVD-CWE-Other,The mime_hdr_cmp function in crypto/asn1/asn_m...
6989,CVE-2006-7252,CWE-189,Integer overflow in the calloc function in lib...
6990,CVE-2006-7253,CWE-255,GE Healthcare Infinia II has a default passwor...


In [23]:
import pandas as pd
import operator
from sklearn.preprocessing import LabelEncoder
dataset = df.copy()
cwe_list=list(dataset['cwe_id'])
cwe_set=set(cwe_list)
cwe_dict={}
dataset

,cve_id,cwe_id,descript
0,CVE-2006-0001,CWE-119,Stack-based buffer overflow in Microsoft Publi...
3,CVE-2006-0004,NVD-CWE-Other,Microsoft PowerPoint 2000 in Office 2000 SP3 h...
4,CVE-2006-0005,CWE-119,Buffer overflow in the plug-in for Microsoft W...
5,CVE-2006-0006,CWE-119,Heap-based buffer overflow in the bitmap proce...
6,CVE-2006-0007,CWE-119,"Buffer overflow in GIFIMP32.FLT, as used in Mi..."
...,...,...,...
6987,CVE-2006-7247,CWE-89,SQL injection vulnerability in the Weblinks (c...
6988,CVE-2006-7250,NVD-CWE-Other,The mime_hdr_cmp function in crypto/asn1/asn_m...
6989,CVE-2006-7252,CWE-189,Integer overflow in the calloc function in lib...
6990,CVE-2006-7253,CWE-255,GE Healthcare Infinia II has a default passwor...


In [58]:
df1=dataset.copy()
df1
cwe_list=list(dataset['cwe_id'])
cwe_set=set(cwe_list)
cwe_dict={}

for cwe in cwe_set:
    cwe_dict[cwe]=cwe_list.count(cwe)

cwe_dict_sort=sorted(cwe_dict.items(),key=operator.itemgetter(1),reverse=True)
cwe_dict_sort
cwe_min_count = 20

for cwe in cwe_set:
    if cwe_dict[cwe] < cwe_min_count or cwe=='NVD-CWE-Other':
    # if cwe_dict[cwe] < cwe_min_count:
        df1 = df1[~df1['cwe_id'].isin([cwe])]
df1
encoder=LabelEncoder()
df1['labels']=encoder.fit_transform(df1['cwe_id'])
cwe_labels = list(df1['labels'])
df1

,cve_id,cwe_id,descript,labels
0,CVE-2006-0001,CWE-119,Stack-based buffer overflow in Microsoft Publi...,0
4,CVE-2006-0005,CWE-119,Buffer overflow in the plug-in for Microsoft W...,0
5,CVE-2006-0006,CWE-119,Heap-based buffer overflow in the bitmap proce...,0
6,CVE-2006-0007,CWE-119,"Buffer overflow in GIFIMP32.FLT, as used in Mi...",0
7,CVE-2006-0008,CWE-264,The ShellAbout API call in Korean Input Method...,5
...,...,...,...,...
6983,CVE-2006-7243,CWE-20,PHP before 5.3.4 accepts the \0 character in a...,2
6984,CVE-2006-7244,CWE-399,Memory leak in pngwutil.c in libpng 1.2.13beta...,6
6985,CVE-2006-7245,CWE-399,Monkey's Audio before 4.01b2 allows remote att...,6
6987,CVE-2006-7247,CWE-89,SQL injection vulnerability in the Weblinks (c...,8


In [59]:
import re
from nltk.stem import WordNetLemmatizer


def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    lemmatizer = WordNetLemmatizer()
#     lem_words = [lemmatizer.lemmatize(w) for w in words]
    lem_words = [lemmatizer.lemmatize(w, pos='n') for w in words]
    lem_words = [lemmatizer.lemmatize(w, pos='v') for w in lem_words]
    stopwords = {}.fromkeys([line.rstrip() for line in open('F:\\PycharmProjects\\NVDproject\\nvdcve\\stopwords.txt')])
    eng_stopwords = set(stopwords)
    words = [w for w in lem_words if w not in eng_stopwords]
    return words

df1['descript'] = df1['descript'].apply(clean_text)
df1

,cve_id,cwe_id,descript,labels
0,CVE-2006-0001,CWE-119,"[stack, base, buffer, overflow, microsoft, pub...",0
4,CVE-2006-0005,CWE-119,"[buffer, overflow, plug, microsoft, window, me...",0
5,CVE-2006-0006,CWE-119,"[heap, base, buffer, overflow, bitmap, process...",0
6,CVE-2006-0007,CWE-119,"[buffer, overflow, gifimp, flt, microsoft, off...",0
7,CVE-2006-0008,CWE-264,"[shellabout, api, call, korean, input, method,...",5
...,...,...,...,...
6983,CVE-2006-7243,CWE-20,"[php, accept, character, pathname, context, de...",2
6984,CVE-2006-7244,CWE-399,"[memory, leak, pngwutil, libpng, beta, version...",6
6985,CVE-2006-7245,CWE-399,"[monkey, audio, remote, attacker, denial, serv...",6
6987,CVE-2006-7247,CWE-89,"[sql, injection, vulnerability, weblinks, webl...",8


In [39]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('accepts', pos='v')

'accept'